# Enter Statefarm competition

In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [30]:
%cd /home/ubuntu/nbs/solutions/statefarm
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
import pandas as pd

/home/ubuntu/nbs/solutions/statefarm


Characterize data set first

In [31]:
%cd /home/ubuntu/data/statefarm

/home/ubuntu/data/statefarm


In [32]:
%ls -1 train/*/*.jpg | wc

  22424   22424  782853


In [33]:
%ls -1 test/unknown | wc

  79726   79726 2065911


In [34]:
df = pd.read_csv('driver_imgs_list.csv')

In [44]:
print(df.head())

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg


In [48]:
print(df.groupby(['subject']).count())

         classname   img
subject                 
p002           725   725
p012           823   823
p014           876   876
p015           875   875
p016          1078  1078
p021          1237  1237
p022          1233  1233
p024          1226  1226
p026          1196  1196
p035           848   848
p039           651   651
p041           605   605
p042           591   591
p045           724   724
p047           835   835
p049          1011  1011
p050           790   790
p051           920   920
p052           740   740
p056           794   794
p061           809   809
p064           820   820
p066          1034  1034
p072           346   346
p075           814   814
p081           823   823


How to split test and validation sets based on subject/driver
There are n = 26 drivers and approximately 22K images.
~ 10% of the data for validation is data for 2-3 drivers
Move data for 3 drivers picked at random p002, p049, p061

In [52]:
subjects = np.random.permutation(df.ix[:,'subject'].unique())
print(subjects[:3])

['p072' 'p016' 'p012']


In [65]:
print(df[df['subject'].isin(subjects[:3])].ix[:,['classname', 'img']].head())

    classname            img
725        c0  img_10206.jpg
726        c0  img_27079.jpg
727        c0  img_50749.jpg
728        c0  img_97089.jpg
729        c0  img_37741.jpg


In [66]:
%cd train

/home/ubuntu/data/statefarm/train


In [68]:
%mkdir ../valid

In [69]:
for d in glob('*'):
    print(d)
    os.mkdir('../valid/' + d)

c7
c8
c6
c0
c2
c4
c9
c1
c3
c5


In [84]:
val_imgs = df[df['subject'].isin(subjects[:3])].ix[:,['img']].values
val_cls = df[df['subject'].isin(subjects[:3])].ix[:,['classname']].values
print(len(val_imgs))
print(len(val_cls))
print(val_imgs.item(1))

2247
2247
img_27079.jpg


In [87]:
for i in range(2247):
    #print(val_cls.item(i))
    os.rename(val_cls.item(i) + '/' + val_imgs.item(i), '../valid/' + val_cls.item(i) + '/' + val_imgs.item(i))

### Create sample

In [89]:
from shutil import copyfile

In [90]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [91]:
for d in glob('*'):
    print(d)
    os.mkdir('../sample/valid/'+d)
    os.mkdir('../sample/train/'+d)

c7
c8
c6
c0
c2
c4
c9
c1
c3
c5


In [98]:
%cd /home/ubuntu/data/statefarm/train
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(3000): copyfile(shuf[i], '../sample/train/'+shuf[i])

/home/ubuntu/data/statefarm/train


In [99]:
%cd ../valid
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(300): copyfile(shuf[i], '../sample/valid/'+shuf[i])

/home/ubuntu/data/statefarm/valid


### Create batches

In [100]:
%cd /home/ubuntu/nbs/solutions/statefarm

/home/ubuntu/nbs/solutions/statefarm


In [101]:
batch_size = 64
#path = '/home/ubuntu/data/statefarm/'
path = '/home/ubuntu/data/statefarm/sample/'
batches = get_batches(path+'train', batch_size=batch_size)

Found 3000 images belonging to 10 classes.


In [102]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 300 images belonging to 10 classes.


In [104]:
test_batches = get_batches('/home/ubuntu/data/statefarm/'+'test', batch_size=batch_size*2, shuffle=False)

Found 79726 images belonging to 1 classes.


#### new VGG16 has batchnorm layers

- use pre-trained VGG weights
- precompute the output of the last convolution layer

In [105]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

553476096/553620808 [============================>.] - ETA: 0s

In [108]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, trn_filenames, test_filenames) = get_classes(path)

Found 3000 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
conv_model = Sequential(conv_layers)
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)